In [ ]:
# Find out the size of vehicle

import numpy as np
import open3d as o3d
from sklearn.cluster import DBSCAN
from scipy.spatial import KDTree

# Load the point cloud data for multiple frames
# cloud_point_data = np.load('/Users/apple/Documents/UniversityDocument/Thesis/automative_self_driving_car/17-10-2024/cloud_point_data2.npy', allow_pickle=True)[300:310]

import pickle
# Load the pickle file
with open('/Users/apple/Documents/UniversityDocument/Thesis/automative_self_driving_car/Autonomous-Navigation-and-Collision-Avoidance-Sytem-for-car/Collision_Avoidance/data/cloud_point_data4k.pkl', 'rb') as f:
    cloud_point_data = pickle.load(f)
    
print(f"Loaded {len(cloud_point_data)} point clouds")
cloud_point_data = cloud_point_data[500:503]
print(f"Loaded {len(cloud_point_data)} point clouds")

# Parameters
num_frames = len(cloud_point_data)
eps = 0.5
min_samples = 10

# Define vehicle size ranges
vehicle_size_ranges = {
    'car': {'0': (1, 4.9), '1': (1.0, 2.0), '2': (1, 1.5)},
    'truck': {'0': (2.5, 6.0), '1': (1, 3.0), '2': (1.0, 3.0)},
    'van': {'0': (2.0, 5.5), '1': (0.5, 2.0), '2': (1.0, 1.8)},
    'motorcycle': {'0': (1.0, 2.5), '1': (0.3, 1.8), '2': (0.4, 1.7)}
}

# [5.047304630279541, 2.0827235281467438, 1.0819239616394043]
#   Cluster Color: yellow --- Bounding Box Dimensions: 0: 5.05, 1: 1.08, 2: 2.08


vehicle_centroid_colors = {
    'car': [0, 0, 1],        # Blue for cars
    'truck': [1, 1, 0],      # Yellow for trucks
    'motorcycle': [0, 1, 0],  # Green for motorcycles
    'van': [0.5, 0.4, 0.4]
}


# Predefined color names and their RGB values
color_names = ['red', 'green', 'blue', 'yellow', 'orange', 'purple', 'cyan', 'magenta', 'pink',
               'brown', 'lime', 'navy', 'olive', 'teal', 'maroon', 'turquoise', 'gold', 'silver',
               'indigo', 'violet']

color_values = {
    'red': [1, 0, 0], 'green': [0, 1, 0], 'blue': [0, 0, 1], 'yellow': [1, 1, 0],
    'orange': [1, 0.5, 0], 'purple': [0.5, 0, 0.5], 'cyan': [0, 1, 1], 'magenta': [1, 0, 1],
    'pink': [1, 0.75, 0.8], 'brown': [0.6, 0.3, 0.1], 'lime': [0.75, 1, 0], 'navy': [0, 0, 0.5],
    'olive': [0.5, 0.5, 0], 'teal': [0, 0.5, 0.5], 'maroon': [0.5, 0, 0], 'turquoise': [0.25, 0.88, 0.82],
    'gold': [1, 0.84, 0], 'silver': [0.75, 0.75, 0.75], 'indigo': [0.29, 0, 0.51], 'violet': [0.93, 0.51, 0.93]
}

# Initialize tracking history (to store cluster IDs and their positions)
cluster_history = []
tracking_id = 0
cluster_color_map = {}  # Map to store consistent colors for clusters
previous_centroids = []  # Track centroids of clusters from the previous frame
previous_cluster_ids = []  # Track cluster IDs from the previous frame

def track_clusters(current_centroids, previous_centroids, prev_cluster_ids, threshold=1.0):
    """Match current centroids with previous centroids using nearest neighbor approach."""
    if len(previous_centroids) == 0:
        return np.arange(len(current_centroids)) + tracking_id, tracking_id + len(current_centroids)
    
    # Create a KDTree for fast nearest neighbor lookup
    tree = KDTree(previous_centroids)
    
    # Assign new tracking IDs based on nearest neighbors from the previous frame
    assigned_ids = np.full(len(current_centroids), -1)  # -1 for new clusters
    distances, indices = tree.query(current_centroids)
    
    # Assign tracking IDs based on distance threshold
    for i, (dist, idx) in enumerate(zip(distances, indices)):
        if dist < threshold:
            assigned_ids[i] = prev_cluster_ids[idx]  # Match with existing cluster ID
    
    # Assign new IDs to unmatched clusters
    new_clusters = (assigned_ids == -1)
    assigned_ids[new_clusters] = np.arange(tracking_id, tracking_id + np.sum(new_clusters))
    
    return assigned_ids, tracking_id + np.sum(new_clusters)

def classify_vehicle(bbox):
    """Classifies the object as a car, truck, or motorcycle based on the bounding box dimensions."""
    dimensions = bbox.get_extent() # Sort to get length, width, height in a consistent order
    dimensions = sorted(dimensions, reverse=True)
    print(dimensions)

    d0, d1, d2 = dimensions[0], dimensions[1], dimensions[2]

    for vehicle_type, size_range in vehicle_size_ranges.items():
        if (size_range['0'][0] <= d0 <= size_range['0'][1] and
            size_range['1'][0] <= d1 <= size_range['1'][1] and
            size_range['2'][0] <= d2 <= size_range['2'][1]):
            return vehicle_type
    return None

for frame_idx in range(num_frames):
    # DBSCAN clustering for the current frame
    current_frame = cloud_point_data[frame_idx]
    clustering = DBSCAN(eps=eps, min_samples=min_samples).fit(current_frame)
    labels = clustering.labels_

    # Extract cluster centroids for the current frame
    current_centroids = []
    cluster_points = []
    
    for cluster_label in set(labels):
        if cluster_label == -1:
            continue  # Skip noise points
        
        cluster_data = current_frame[labels == cluster_label]
  
        if np.any(cluster_data[:, 2] >= 2):
            continue
        
        cluster_points.append(current_frame[labels == cluster_label])
        cluster_centroid = np.mean(current_frame[labels == cluster_label], axis=0)
        current_centroids.append(cluster_centroid)
    
    current_centroids = np.array(current_centroids)
    
    # Track clusters by matching current centroids with previous centroids
    if frame_idx == 0:
        # For the first frame, initialize the cluster IDs
        cluster_ids = np.arange(len(current_centroids))
        tracking_id = len(current_centroids)
    else:
        cluster_ids, tracking_id = track_clusters(current_centroids, previous_centroids, previous_cluster_ids)

    # Save the cluster history for visualization
    cluster_history.append((cluster_points, current_centroids, cluster_ids))
    
    # Update previous frame's centroids and IDs for the next iteration
    previous_centroids = current_centroids
    previous_cluster_ids = cluster_ids

    # Assign unique colors to clusters persistently across frames (tracking colors)
    for cluster_id in cluster_ids:
        if cluster_id not in cluster_color_map:
            color_index = len(cluster_color_map) % len(color_names)  # Cycle through the color list
            cluster_color_map[cluster_id] = (color_names[color_index], color_values[color_names[color_index]])

# Visualization across multiple frames with consistent colors for tracked clusters
for frame_idx, (cluster_points, centroids, cluster_ids) in enumerate(cluster_history):
    print(f"\nFrame {frame_idx}:")
    all_points = []
    all_colors = []
    bounding_boxes = []
    centroids_spheres = []
    
    for i, cluster in enumerate(cluster_points):
        # Add all points to the list for visualization
        all_points.append(cluster)
        
        # Create an Open3D point cloud for the current cluster
        cluster_pcd = o3d.geometry.PointCloud()
        cluster_pcd.points = o3d.utility.Vector3dVector(cluster)
        
        # Compute the bounding box for the cluster
        bbox = cluster_pcd.get_axis_aligned_bounding_box()
        dimensions = bbox.get_extent()
        dimension_text = f"0: {dimensions[0]:.2f}, 1: {dimensions[1]:.2f}, 2: {dimensions[2]:.2f}"
        
        # Get cluster color for tracking (original color for points)
        cluster_color_name, cluster_color_value = cluster_color_map[cluster_ids[i]]
        
        # Add color for visualization
        all_colors.append(np.tile(cluster_color_value, (len(cluster), 1)))  # Use tracking color for points
        
        # Add the bounding box for visualization
        bbox.color = (0, 1, 0)  # Set bounding box color to green
        bounding_boxes.append(bbox)
        
        # Use bounding box center as centroid and classify for vehicle types
        bbox_center = bbox.get_center()
        vehicle_type = classify_vehicle(bbox)
        
        if vehicle_type:
            centroid_color = vehicle_centroid_colors[vehicle_type]  # Color based on vehicle type
            print(f" Cluster Color: {cluster_color_name}  Detected a {vehicle_type.upper()} with bounding box dimensions: {sorted(bbox.get_extent(), reverse=True)}")
        else:
            centroid_color = [1, 0, 0]  # Default red if no vehicle classification
            
        print(f"  Cluster Color: {cluster_color_name} --- Bounding Box Dimensions: {dimension_text}")
        
        # Create a sphere at the centroid with the appropriate vehicle type color
        sphere = o3d.geometry.TriangleMesh.create_sphere(radius=0.2)
        sphere.translate(bbox_center)
        sphere.paint_uniform_color(centroid_color)  # Set centroid color based on vehicle type
        centroids_spheres.append(sphere)

    if len(all_points) > 0:
        # Flatten the points and colors arrays for visualization
        all_points = np.vstack(all_points)
        all_colors = np.vstack(all_colors)
        
        # Create an Open3D point cloud object for visualization
        pcd = o3d.geometry.PointCloud()
        pcd.points = o3d.utility.Vector3dVector(all_points)
        pcd.colors = o3d.utility.Vector3dVector(all_colors)
    
        # Visualize all points with bounding boxes, tracked cluster colors, and red centroids
        o3d.visualization.draw_geometries([pcd] + bounding_boxes + centroids_spheres,
                                          zoom=0.4,
                                          front=[1.0, -1.0, 0.5],  # Set the side view of the camera
                                          lookat=[0.0, 0.0, 0.0],
                                          up=[0.0, 0.0, 1.0])



Loaded 4625 point clouds
Loaded 3 point clouds
